In [1]:
from transformers import RemBertTokenizer, RemBertForTokenClassification
from tqdm.notebook import tqdm


# Load RemBERT and tokenizer
#tokenizer = RemBertTokenizer.from_pretrained('google/rembert')
#model = RemBertForTokenClassification.from_pretrained('google/rembert', num_labels=9)

/home/sginne/anaconda3/envs/flair_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import BertTokenizer, BertForTokenClassification

# Load BERT and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=9)


/home/sginne/anaconda3/envs/flair_env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
!pip install flair

import requests
from flair.datasets import ColumnCorpus
import os

# Define function to download data
def download_data(url, file_name):
    response = requests.get(url)
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(response.text)

# Define URLs
train_url = 'https://raw.githubusercontent.com/lang-uk/flair-ner/main/fixed-split/train.iob'
test_url = 'https://raw.githubusercontent.com/lang-uk/flair-ner/main/fixed-split/test.iob'

# Download train and test datasets
data_folder = './data/'
os.makedirs(data_folder, exist_ok=True)
download_data(train_url, os.path.join(data_folder, 'train.iob'))
download_data(test_url, os.path.join(data_folder, 'test.iob'))

# Define columns - typically: 0: token, 1: NER tag
columns = {0: 'text', 1: 'ner'}

# Initialize the corpus using your downloaded train and test files
corpus = ColumnCorpus(data_folder, columns,
                      train_file='train.iob',
                      test_file='test.iob')
#corpus = corpus.downsample(0.1)

# Print some stats about the dataset

#print(corpus)
print("Train:")
for i in range(5):
    print(corpus.train[i])

print("\nTest:")
for i in range(5):
    print(type(corpus.test[i]))


2024-09-07 13:28:01,567 Reading data from data
2024-09-07 13:28:01,568 Train: data/train.iob
2024-09-07 13:28:01,569 Dev: None
2024-09-07 13:28:01,569 Test: data/test.iob
2024-09-07 13:28:03,548 No dev split found. Using 10% (i.e. 876 samples) of the train split as dev data
Train:
Sentence[18]: "Зрозуміло , що український бізнес почав використовувати КСВ як інструмент формування своєї репутації буквально декілька років тому ."
Sentence[22]: "З одного боку , саме через це більшість проектів КСВ здійснюються епізодично та деколи виглядають , радше , як просто благодійність ."
Sentence[8]: "Винятком будуть хіба що представництва іноземних корпорацій ."
Sentence[20]: "З іншого боку , для українських компаній відкривається потужне « вікно можливостей » , щоб втілювати власні унікальні стратегії ."
Sentence[3]: "Філантропія як пріоритет"

Test:
<class 'flair.data.Sentence'>
<class 'flair.data.Sentence'>
<class 'flair.data.Sentence'>
<class 'flair.data.Sentence'>
<class 'flair.data.Sentence'>

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"



In [5]:
print(os.getenv("PYTORCH_CUDA_ALLOC_CONF"))


expandable_segments:True


In [21]:
tag_dict = corpus.make_tag_dictionary(tag_type='ner')
print("TAGS:", tag_dict)
print("Tags:", tag_dict.get_items())

/tmp/ipykernel_1135083/2275610158.py:1: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  tag_dict = corpus.make_tag_dictionary(tag_type='ner')


TAGS: Dictionary with 3 tags: O, <START>, <STOP>
Tags: ['O', '<START>', '<STOP>']


In [20]:
with open(os.path.join(data_folder, 'train.iob'), 'r', encoding='utf-8') as f:
    lines = f.readlines()
    print("Sample from train.iob:")
    for line in lines[110:250]:  # Print first 10 lines
        print(line.strip())

Sample from train.iob:
власна O
ініціатива O
, O
так O
і O
відповідь O
на O
запит O
від O
громади O
. O


Це O
абсолютно O
нормальна O
практика O
через O
недостатній O
рівень O
статків O
та O
доходів O
переважної O
більшості O
українських O
громадян O
та O
громад O
. O


І O
хоча O
деякі O
місцеві O
експерти O
вважають O
це O
базовим O
рівнем O
, O
що O
свідчить O
про O
недостатній O
розвиток O
КСВ O
, O
ця O
діяльність O
заслуговує O
на O
повагу O
. O


Особливо O
якщо O
зауважити O
, O
що O
такі O
проекти O
часто O
спрямовані O
на O
обладнання O
шкіл O
, O
сприяння O
освіті O
, O
допомогу O
вимушеним O
переселенцям O
або O
літнім O
людям O
, O
як O
, O
наприклад O
, O
проект O
« O
Життєлюб B-ORG
» O
Гаріка B-PERS
Корогодського I-PERS
. O


Вони O
абсолютно O
відповідають O
Глобальному B-MISC
договору I-MISC
та I-MISC
Цілям I-MISC
сталого I-MISC
розвитку I-MISC
ООН I-MISC
. O





Виклики O
воєнного O
часу O





Підтримка O
Збройних B-ORG
Сил I-ORG
— O
один O
з O
унікальних O
і O
сут

In [15]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

# Load RemBERT as the transformer embedding
#rembert_embeddings = TransformerWordEmbeddings('google/rembert', fine_tune=True)
distilbert_embeddings = TransformerWordEmbeddings('distilbert-base-multilingual-cased', fine_tune=True)
print(f"TAGS:{corpus.make_tag_dictionary(tag_type='ner')}")

# Create the NER tagger
tagger = SequenceTagger(hidden_size=128,
                        embeddings=distilbert_embeddings,
                        tag_dictionary=corpus.make_tag_dictionary(tag_type='ner'),
                        tag_type='ner',
                        use_crf=False)
import torch
#device = torch.device('cpu')
#tagger.to(device)

# Initialize the trainer
trainer = ModelTrainer(tagger, corpus)
#print(f'Corpus:{corpus}\nTagger:{tagger}\nModelTrainer:{trainer}')


/tmp/ipykernel_1135083/2829087065.py:8: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  print(f"TAGS:{corpus.make_tag_dictionary(tag_type='ner')}")


TAGS:Dictionary with 3 tags: O, <START>, <STOP>


/tmp/ipykernel_1135083/2829087065.py:13: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  tag_dictionary=corpus.make_tag_dictionary(tag_type='ner'),


2024-09-07 13:21:01,374 SequenceTagger predicts: Dictionary with 3 tags: O, <START>, <STOP>


In [7]:
import torch
torch.cuda.empty_cache()

trainer.train('saved_model2',
              learning_rate=0.001,
              mini_batch_size=4,
              max_epochs=2,
              embeddings_storage_mode='cpu',
              #main_evaluation_metric='f1',
              monitor_test=True,
              #use_amp=True,
              train_with_dev=True)  # Enable logging to file



2024-09-07 12:57:14,081 ----------------------------------------------------------------------------------------------------
2024-09-07 12:57:14,082 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(119548, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_featu

/home/sginne/anaconda3/envs/flair_env/lib/python3.8/site-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-09-07 12:57:38,547 epoch 1 - iter 219/2191 - loss 0.38020160 - time (sec): 24.42 - samples/sec: 635.72 - lr: 0.001000 - momentum: 0.000000
2024-09-07 12:58:00,916 epoch 1 - iter 438/2191 - loss 0.19157820 - time (sec): 46.79 - samples/sec: 667.92 - lr: 0.001000 - momentum: 0.000000
2024-09-07 12:58:23,020 epoch 1 - iter 657/2191 - loss 0.12745257 - time (sec): 68.90 - samples/sec: 683.70 - lr: 0.001000 - momentum: 0.000000
2024-09-07 12:58:45,734 epoch 1 - iter 876/2191 - loss 0.09513947 - time (sec): 91.61 - samples/sec: 690.58 - lr: 0.001000 - momentum: 0.000000
2024-09-07 12:59:07,293 ----------------------------------------------------------------------------------------------------
2024-09-07 12:59:07,294 Exiting from training early.
2024-09-07 12:59:07,295 Saving model ...
2024-09-07 12:59:12,700 Done.
2024-09-07 12:59:12,755 ----------------------------------------------------------------------------------------------------
2024-09-07 12:59:12,757 Testing using last state o

 14%|█▍        | 9/64 [00:10<00:47,  1.15it/s]

In [ ]:
result = tagger.evaluate(corpus.test, gold_label_type='ner')  # Evaluate on the test set
print(result.detailed_results)  # Print detailed evaluation results


100%|██████████| 127/127 [00:50<00:00,  2.52it/s]


Results:
- F-score (micro) 0.0
- F-score (macro) 0.0
- Accuracy 0.0

By class:
              precision    recall  f1-score   support

        PERS     0.0000    0.0000    0.0000    1678.0
         LOC     0.0000    0.0000    0.0000     401.0
         ORG     0.0000    0.0000    0.0000     261.0
        MISC     0.0000    0.0000    0.0000     240.0

   micro avg     0.0000    0.0000    0.0000    2580.0
   macro avg     0.0000    0.0000    0.0000    2580.0
weighted avg     0.0000    0.0000    0.0000    2580.0



In [ ]:
print(corpus.train[0])  # Print the first sample from the training data
print(corpus.test[0])   # Print the first sample from the test data


Sentence[18]: "Зрозуміло , що український бізнес почав використовувати КСВ як інструмент формування своєї репутації буквально декілька років тому ."
Sentence[26]: "Цікаву і пізнавальну подорож місцями , де народився та виріс видатний український поет Тарас Шевченко , минулого тижня влаштував департамент культури і туризму Кіровоградської ОДА ." → ["Тарас Шевченко"/PERS, "Кіровоградської ОДА"/ORG]


In [ ]:
from flair.models import SequenceTagger
from flair.data import Sentence
from tqdm.notebook import tqdm


#if model is None:
model = SequenceTagger.load('saved_model2/final-model.pt')
# Function to predict and print tagged sentences
def print_tagged_sentences(sentences, model=None):
    # Load model only if not already loaded
    
    for text in sentences:
        sentence = Sentence(text)  # Create Sentence object
        model.predict(sentence)    # Predict tags
        print("TAGGED:",sentence.to_tagged_string())  # Print tagged sentence

# Example usage
sentences = ["Привіт, мене звати Тімо і я живу в Турку, навчаюсь в Универсітеті Турку.",
"Цікаву і пізнавальну подорож місцями , де народився та виріс видатний український поет Тарас Шевченко , минулого тижня влаштував департамент культури і туризму Кіровоградської ОДА ."]
print_tagged_sentences(sentences, model)



2024-09-07 12:54:01,081 SequenceTagger predicts: Dictionary with 3 tags: O, <START>, <STOP>
TAGGED: Sentence[16]: "Привіт, мене звати Тімо і я живу в Турку, навчаюсь в Универсітеті Турку."
TAGGED: Sentence[26]: "Цікаву і пізнавальну подорож місцями , де народився та виріс видатний український поет Тарас Шевченко , минулого тижня влаштував департамент культури і туризму Кіровоградської ОДА ."
